# Custom functionals


### PySCF
Builtin functional, custom functional, CCSD(T)

### DQC 
Custom functional 

### Finite Differences and Autograd

Bump parameters in dqc and pyscf and then test with autograd


# Adjoint Methods with DQC

### Implementation in DQC (example with one molecule)

implementation details, adjoint equations, problem with 4-term integral, adjoint orthogonality 

### Comparison with finite-differences on several molecules

adjoint orthogonality holds and coherence with finite differences


# Optimal Implementation in PySCF

Benchmark across all molecules against finite diff (pyscf) and autograd (dqc)